In [7]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import IPython.display as ipd
import plotly.express as px
import librosa.display
import pandas as pd
import numpy as  np
import librosa
import cv2
import warnings
import IPython
import os
import glob
from tqdm import tqdm

from PIL import Image
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

plt.style.use("ggplot")

### Read data

In [2]:
train = pd.read_csv("./rawdata/train.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21375 entries, 0 to 21374
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rating            21375 non-null  float64
 1   playback_used     19575 non-null  object 
 2   ebird_code        21375 non-null  object 
 3   channels          21375 non-null  object 
 4   date              21375 non-null  object 
 5   pitch             21375 non-null  object 
 6   duration          21375 non-null  int64  
 7   filename          21375 non-null  object 
 8   speed             21375 non-null  object 
 9   species           21375 non-null  object 
 10  number_of_notes   21375 non-null  object 
 11  title             21375 non-null  object 
 12  secondary_labels  21375 non-null  object 
 13  bird_seen         19575 non-null  object 
 14  sci_name          21375 non-null  object 
 15  location          21375 non-null  object 
 16  latitude          21375 non-null  object

In [3]:
species=train.species.value_counts()
birds=train.ebird_code.unique()

In [8]:
def create_split_stfit_datasets(birds, species, interval=5):
    audio_dir_path="./rawdata/train_audio/"
    for bird_name in tqdm(birds):
        print("---------------- {} ----------------".format(bird_name))
        if not os.path.exists(os.path.join('data', bird_name)):
            os.mkdir(os.path.join('data', bird_name))
        files = train[train.ebird_code==bird_name]['filename'].values
        for file in tqdm(files, leave=False):
            audio_path=os.path.join(audio_dir_path,bird_name,file)
            x , sr = librosa.load(audio_path)
            print("Audio Length: {}".format(len(x) / sr))
            i = 0
            while ((i+interval) * sr < len(x)):
                sub_x = x[i*sr:(i+interval)*sr]
                sub_x = librosa.stft(sub_x)
                Xdb = librosa.amplitude_to_db(abs(sub_x))
                np.savetxt('./data/{}/{}_{}_{}to{}sec.csv'.format(bird_name, bird_name, file.split('.')[0], i, i+interval), Xdb)
                i = i + interval

interval = 5
create_split_stfit_datasets(birds, species, interval)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]c:\users\owner\appdata\local\programs\python\python38\lib\site-packages\librosa\core\audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


---------------- aldfly ----------------
Audio Length: 25.488027210884354



  1%|▊                                                                                 | 1/100 [00:01<03:04,  1.87s/it]

Audio Length: 36.310204081632655



  2%|█▋                                                                                | 2/100 [00:04<03:22,  2.07s/it]

...

Audio Length: 71.23591836734694



  8%|██████▌                                                                           | 8/100 [00:20<04:38,  3.02s/it]

Audio Length: 26.253061224489795



  0%|                                                                                          | 0/264 [00:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
#画像のサイズ指定
ScaleTo = 70
seed = 7
n_categories = 12

In [ ]:
#トレーニングデータの読み込み
data_dir ="../input/train"
path = "../input/train/*/*.png"
files = glob(path)

In [ ]:
trainImg = []
trainLabel = []
j = 1
num = len(files)


In [ ]:
#画像データをリストに格納
for img in files:
    print(str(j) + "/" + str(num) , end="\r")
    trainImg.append(cv2.resize(cv2.imread(img) ,(ScaleTo,ScaleTo)))
    #trainLabel.append(img.split("/")[-2])
    j += 1

In [ ]:
#ラベルをリストに格納
for dir in os.listdir(data_dir):
    if dir == ".DS_Store":
        continue
    dir1 = data_dir + "/" + dir
    label = dir

    for file in os.listdir(dir1):
        if file != "Thumbs.db":

            trainLabel.append(label)

In [ ]:
# kerasに渡すためにnumpy配列に変換。
image_list = np.asarray(trainImg)
label_list = pd.DataFrame(trainLabel)

In [ ]:
clearTrainImg = []
examples = [];getEx = True

for img in image_list:
    #ぼかしを入れてノイズを除去
    blurImg = cv2.GaussianBlur(img ,(5,5),0)
    #RGBからHSVに変換
    hsvImg = cv2.cvtColor(blurImg , cv2.COLOR_BGR2HSV)
    #マスクを作成
    lower_green = (25,40,50)
    upper_green = (75,255,255)
    mask = cv2.inRange(hsvImg , lower_green ,upper_green)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
    mask = cv2.morphologyEx(mask , cv2.MORPH_CLOSE,kernel)

    #ブールマスクの作成
    bMask = mask > 0

    #マスクの適用
    #空のイメージの作成
    clear = np.zeros_like(img , np.uint8)
    #オリジナル画像にブールマスクを適用
    clear[bMask] = img[bMask]

    clearTrainImg.append(clear)

In [ ]:
for i in range(8):
    plt.subplot(2, 4, i + 1)
    plt.imshow(clearTrainImg[i])

In [ ]:
clearTrainImg = np.asarray(clearTrainImg)
clearTrainImg.shape
# クラスの形式を変換
le = LabelEncoder()
le = le.fit(label_list)
label_list = le.transform(label_list)
label_list
label_list = np_utils.to_categorical(label_list)
label_list

X_train, X_test, y_train, y_test = train_test_split(clearTrainImg, label_list, test_size=0.33, random_state=0)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        rotation_range=180,  # randomly rotate images in the range
        zoom_range = 0.1, # Randomly zoom image
        width_shift_range=0.1,  # randomly shift images horizontally
        height_shift_range=0.1,  # randomly shift images vertically
        horizontal_flip=True,  # randomly flip images horizontally
        vertical_flip=True  # randomly flip images vertically
    )
datagen.fit(X_train)

In [ ]:
#モデル作成・学習
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import tensorflow as tf
import keras
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.layers import Dense, Dropout, Flatten, Activation,GlobalAveragePooling2D,Input
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras import optimizers


# ResNet50のロード。FC層は不要なので include_top=False
input_tensor = Input(shape=(ScaleTo, ScaleTo, 3))
resnet50 = ResNet50(include_top=False, weights='imagenet', input_tensor=input_tensor)

# FC層の作成
top_model = Sequential()
top_model.add(Flatten(input_shape=resnet50.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(nb_classes, activation='softmax'))

# ResNet50とFC層を結合してモデルを作成
resnet50_model = Model(input=resnet50.input, output=top_model(resnet50.output))

#ResNet50の一部の重みを固定
for layer in resnet50_model.layers[:100]:
    layer.trainable = False

# 多クラス分類を指定
resnet50_model.compile(loss='categorical_crossentropy',
          optimizer=optimizers.SGD(lr=1e-3, momentum=0.9),
          metrics=['accuracy'])
resnet50_model.summary()


#学習の実行
hist = resnet50_model.fit_generator(datagen.flow(X_train, y_train, batch_size=75),
                        epochs=35, validation_data=(X_test, y_test),
                        steps_per_epoch=X_train.shape[0])


                        steps_per_epoch=X_train.shape[0])


#モデルの評価
print(resnet50_model.evaluate(X_train, y_train)) #トレーニングの精度
print(resnet50_model.evaluate(X_test, y_test))  #テスト精度


#パラメータの保存
model.save_weights('../content/weights.h5')